In [1]:
import os
import tqdm
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import Set
import Utility
import SQLSentence
import Model

%load_ext autoreload
%autoreload 1
%aimport Model
%aimport Set
%aimport Utility

In [2]:
# Build Environment
connect = Utility.connect_to_database()
main_data, list_stock_type, dict_code_name, dict_type_name = Utility.fetch_data_from_db()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Constants
PATH_CHECKPOINT = 'checkpoint'

if not os.path.exists(PATH_CHECKPOINT):
    os.makedirs(PATH_CHECKPOINT)

myseed = 42069  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

In [21]:
# Dataset
class StockDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.shape = data.shape

        print(label.shape)

    def __getitem__(self, index):
        data = torch.tensor(self.data[index], dtype=torch.float32)
        label = torch.tensor(self.label[index], dtype=torch.float32)

        return data, label
    
    def __len__(self):
        return len(self.data)

In [22]:
# Some Functions
def process_stock_data(stock_data, target, window_length, predict = False):
    stock_data = stock_data.drop(['code', 'date'], axis=1).reset_index(drop=True)
    stock_data = Set.Feature(stock_data)
    stock_data = stock_data.dropna()

    index_col_open = stock_data.columns.get_loc("open")
    index_col_high = stock_data.columns.get_loc("high")
    index_col_low = stock_data.columns.get_loc("low")
    index_col_volume = stock_data.columns.get_loc("volume")

    stock_data = stock_data.to_numpy()

    data_merged = []
    label_merged = []

    if predict:
        return stock_data[-window_length:, index_col_volume + 1:]

    for i in range(len(stock_data) - window_length):
        data = stock_data[i:i + window_length, index_col_volume + 1:]

        if target >= 0:
            target_value = (stock_data[i + window_length, index_col_high] - stock_data[i + window_length, index_col_open]) / \
                stock_data[i + window_length, index_col_open]
            label = int(target_value >= target)
        else:
            target_value = (stock_data[i + window_length, index_col_low] - stock_data[i + window_length, index_col_open]) / \
                stock_data[i + window_length, index_col_open]
            label = int(target_value <= target)
        
        # window_data_with_label = np.append(window_data, label)

        data_merged.append(data)
        label_merged.append(label)

    return np.array(data_merged), np.array(label_merged)

def build_dataset(main_data, code_list, target, window_length):
    all_train_data = []
    all_test_data = []

    all_train_label = []
    all_test_label = []

    for code in code_list:
        stock_data = main_data[main_data['code'] == code].copy()

        data, label = process_stock_data(stock_data, target, window_length)
        train_size = int(0.9 * len(data))
               
        train_data = data[:train_size]
        test_data = data[train_size:]

        train_label = label[:train_size]
        test_label = label[train_size:]

        all_train_data.extend(train_data)
        all_test_data.extend(test_data)

        all_train_label.extend(train_label)
        all_test_label.extend(test_label)
    
    return np.array(all_train_data), np.array(all_test_data), np.array(all_train_label), np.array(all_test_label)

def build_dataloader(main_data, code_list, target, batch_size, window_length):
    train_data, test_data, train_label, test_label = build_dataset(main_data, code_list, target, window_length)
    dataset_train = StockDataset(train_data, train_label)
    dataset_test = StockDataset(test_data, test_label)
    datalaoder_train = DataLoader(dataset_train, batch_size, shuffle=True)
    datalaoder_test = DataLoader(dataset_test, batch_size, shuffle=False)

    return datalaoder_train, datalaoder_test

def calculate_correct_count(predicted, label):
    type1_correct = 0
    type2_correct = 0
    type1_count = 0
    type2_count = 0

    for i in range(label.size(0)):
        # Type1
        if label[i] == 1:
            type1_count += 1
            if predicted[i] == 1:
                type1_correct += 1

        # Type2
        elif label[i] == 0:
            type2_count += 1
            if predicted[i] == 0:
                type2_correct += 1

    return type1_correct, type2_correct, type1_count, type2_count

def predict_result(main_data, code, model, device, target, window_length, min_period=100):
    stock_data = main_data[main_data['code'] == code].tail(min_period).copy()
    data, _ = process_stock_data(stock_data, target, window_length, predict=True)
    output = model(torch.tensor(data, dtype=torch.float32).reshape(1, data.shape[0]).to(device))
    return round(output[0].cpu().detach().item(), 4)

In [26]:
# Invoke Model
def invoke_model(mode, model, dataloader, criterion, device, optimizer=None):
    
    model.train() if mode == 'train' else model.eval()

    total_loss = 0
    correct, accuracy_count = 0, 0
    investable_count, not_investable_count = 0, 0
    actual_investable_count, actual_not_investable_count = 0, 0
    type1_correct, type2_correct = 0, 0
    type1_count, type2_count = 0, 0

    for input, label in tqdm.tqdm(dataloader):
        input, label = input.to(device), label.to(device)

        if mode == 'train' and optimizer is not None:
            optimizer.zero_grad()
        
        outputs = model(input)
        loss = criterion(outputs.squeeze(1), label)
        total_loss+=loss.detach().cpu().item()

        predicted = (outputs.squeeze(1) > 0).float()

        if mode == 'train' and optimizer is not None:
            loss.backward()
            optimizer.step()

        correct += (predicted == label).sum().item()
        accuracy_count += label.size(0)

        investable_count += (predicted == 1).sum().item()
        not_investable_count += (predicted == 0).sum().item()

        actual_investable_count += (label == 1).sum().item()
        actual_not_investable_count += (label == 0).sum().item()

        t1_correct, t2_correct, t1_cnt, t2_cnt = calculate_correct_count(predicted, label)
        type1_correct += t1_correct
        type2_correct += t2_correct
        type1_count += t1_cnt
        type2_count += t2_cnt

    print(f'\n預測可投資的次數: {investable_count} / 實際上預測可投資的次數: {actual_investable_count}')
    print(f'不可投資的次數: {not_investable_count} / 實際上不可投資的次數: {actual_not_investable_count}')

    avg_loss = round(total_loss/len(dataloader), 6)
    accuracy =  round((correct/accuracy_count)*100 , 2)
    type1_correct_ratio = round((type1_correct/type1_count)*100, 2)
    type2_correct_ratio = round((type2_correct/type2_count)*100, 2)

    print(f'\n{mode} Loss: {avg_loss}')
    print(f"Accuracy: {accuracy}%")
    print(f"\nType1 Correct Ratio: {type1_correct_ratio}%")
    print(f"Type2 Correct Ratio: {type2_correct_ratio}%\n")

    return avg_loss, type1_correct_ratio, type2_correct_ratio

In [ ]:
# Get data from single Type
BATCH_SIZE = 256
N_EPOCHS = 1

# [0, 7, 8, 13, 17, 18, 24, 25, 26, 27, 30, 31, 32, 35, 37]
type_id=7

target_pct = 0.02 # Target rise percentage value
window_length = 20

list_unique_code_from_data = (SQLSentence.GetCodeByTypeId(type_id, connect))['code'].tolist()
datalaoder_train, datalaoder_test = build_dataloader(main_data, list_unique_code_from_data, target_pct, BATCH_SIZE, window_length)

In [ ]:
# Select single Type to adjust the model and the optimizer
lr = Set.GetInfo('learning_rate')

model = Model.CNN_LSTM(datalaoder_train.dataset.shape).to(device)
type_id_pos_weight = (Set.GetInfo('pos_weight'))[dict_type_name[type_id]]
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([type_id_pos_weight]).to(device))
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

threshold=0.5
best_test_loss = Utility.load_best_model_record_txt(type_id, target_pct, model.__class__.__name__)
train_avg_loss = 0
test_avg_loss = 0

use_checkpoint = False
if use_checkpoint:
    try:
        postfix='save'
        model.load_state_dict(torch.load(f"{PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_{postfix}", weights_only=True))
    except:
        print(f'No preserved model: {model.__class__.__name__}')

epoch = 0
while epoch < N_EPOCHS:
    epoch+=1
    print(f'Train {dict_type_name[type_id]} Epoch: {epoch}')

    # Train
    train_avg_loss, type1_correct_ratio, type2_correct_ratio = invoke_model('train', model, datalaoder_train, criterion, device, optimizer)
    print('--------------------------------------------------------------------------------------------------------------------------------------------')

    # Test
    with torch.no_grad():
        test_avg_loss, _, _ = invoke_model('test', model, datalaoder_test, criterion, device)

    # Save the model with a best loss (test) using postfix: best
    if test_avg_loss < best_test_loss:
        best_test_loss = test_avg_loss
        print(f'Model has saved in {PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_best\n')
        # torch.save(model.state_dict(), f'{PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_best')
        # Utility.update_model_record_txt(type_id, target_pct, model.__class__.__name__, best_test_loss)

    print('============================================================================================================================================\n')

# Save the last epoch model using postfix: save
print(f'Model has saved in {PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_save\n')
# torch.save(model.state_dict(), f'{PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_save')

In [ ]:
# Predict
target_pct = 0.02
window_length = 20
threshold=0.5

# list_selected_type_id = [0, 7, 8, 13, 17, 18, 24, 25, 26, 27, 30, 31, 32, 35, 37]
list_selected_type_id = [0]
current_date = (SQLSentence.GetLatestDate(connect)).strftime("%Y-%m-%d")
file_name = f'predict_tmp_{current_date}.txt'

model_name = 'CNN_LSTM'
model_name = 'Model_'+model_name
ModelClass = getattr(Model, model_name)
postfix='best'

if not os.path.exists(file_name):
    with open(file_name, 'w') as f:
        pass

with open(file_name, 'a', encoding='utf-8') as f:
    f.write(f"----------------------------------Model: {model_name}_{postfix}----------------------------")

    for type_id in tqdm.tqdm(list_selected_type_id):

        list_unique_code_from_data = (SQLSentence.GetCodeByTypeId(type_id, connect))['code'].tolist()

        model = ModelClass(datalaoder_test.dataset.dim).to(device)
        model.eval()
        list_code_investable = []

        try:
            model.load_state_dict(torch.load(f"{PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_{postfix}", weights_only=True))
        except:
            print(f'{dict_type_name[type_id]} No preserved model: {model.__class__.__name__}')
            break

        for code in list_unique_code_from_data:
            output = predict_result(main_data, code, model, device, target_pct, window_length)
            if output > threshold:
                list_code_investable.append((code, output))

        if len(list_code_investable) != 0:
            f.write("------------------------------------------------------------------------")
            f.write(f"\n產業: {dict_type_name[type_id]}\n")
            for code, probability in list_code_investable:
                f.write(f'{code} {dict_code_name[code]:<10} Prob: {probability}\n')
    f.write("\n\n")

In [16]:
# Train all data
BATCH_SIZE = 256
N_EPOCHS = 1

lr = Set.GetInfo('learning_rate')
target_pct = 0.02 # Target rise percentage value
window_length = 20

list_selected_type_id = [0, 7, 8, 13, 17, 18, 24, 25, 26, 27, 30, 31, 32, 35, 37]
use_checkpoint = True
postfix='best'
# 2024-08-28 10:07
for type_id in list_selected_type_id:
    print('////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////')
    list_unique_code_from_data = (SQLSentence.GetCodeByTypeId(type_id, connect))['code'].tolist()
    datalaoder_train, datalaoder_test = build_dataloader(main_data, list_unique_code_from_data, target_pct, BATCH_SIZE, window_length)
    
    model = Model.Model_CNN_LSTM(datalaoder_train.dataset.dim).to(device)
    type_id_pos_weight = (Set.GetInfo('pos_weight'))[dict_type_name[type_id]]
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([type_id_pos_weight]).to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.5, 0.999))

    threshold=0.5
    best_test_loss = Utility.load_best_model_record_txt(type_id, target_pct, model.__class__.__name__)
    train_avg_loss = 0
    test_avg_loss = 0
    
    if use_checkpoint:
        try:
            model.load_state_dict(torch.load(f"{PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_{postfix}", weights_only=True))
        except:
            print(f'{dict_type_name[type_id]} No preserved model: {model.__class__.__name__}')
            break


    # good_model_predict_count = 1
    epoch = 0
    while epoch < N_EPOCHS:
        epoch+=1
        print(f'\nTrain {dict_type_name[type_id]} Epoch: {epoch}')

        # Train
        train_avg_loss, type1_correct_ratio, type2_correct_ratio = invoke_model('train', model, datalaoder_train, criterion, device, optimizer)

        # Save the model with a better accuary (train), using postfix: selected{good_model_predict_count}
        # if type1_correct_ratio>25.0 and type2_correct_ratio>85.0:
        #     torch.save(model.state_dict(), f'{PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_selected{good_model_predict_count}')
        #     print(f'Model has saved in {PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_selected{good_model_predict_count}\n')
        #     good_model_predict_count+=1
        
        print('--------------------------------------------------------------------------------------------------------------------------------------------')

        # Test
        with torch.no_grad():
            test_avg_loss, _, _ = invoke_model('test', model, datalaoder_test, criterion, device)

        # Save the model with a best loss (test) using postfix: best
        if test_avg_loss < best_test_loss:
            best_test_loss = test_avg_loss
            print(f'Model has saved in {PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_best')
            torch.save(model.state_dict(), f'{PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_best')
            Utility.update_model_record_txt(type_id, target_pct, model.__class__.__name__, best_test_loss)

        print('============================================================================================================================================')

    # Save the last epoch model using postfix: save
    print(f'Model has saved in {PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_save\n')
    torch.save(model.state_dict(), f'{PATH_CHECKPOINT}/type_{type_id}/type_{type_id}_{model.__class__.__name__}_Target_{int(target_pct*100)}_save')

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

Train 電零組 Epoch: 1


100%|██████████| 1544/1544 [09:38<00:00,  2.67it/s]



預測可投資的次數: 44704 / 實際上預測可投資的次數: 100556
不可投資的次數: 350448 / 實際上不可投資的次數: 294596

train Loss: 0.616543
Accuracy: 74.15%

Type1 Correct Ratio: 21.43%
Type2 Correct Ratio: 92.14%

--------------------------------------------------------------------------------------------------------------------------------------------


100%|██████████| 149/149 [00:51<00:00,  2.89it/s]



預測可投資的次數: 5866 / 實際上預測可投資的次數: 8416
不可投資的次數: 32152 / 實際上不可投資的次數: 29602

test Loss: 0.566787
Accuracy: 76.07%

Type1 Correct Ratio: 30.81%
Type2 Correct Ratio: 88.94%

Model has saved in checkpoint/type_0/type_0_Model_CNN_LSTM_Target_2_save

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

Train 航運 Epoch: 1


...


預測可投資的次數: 7223 / 實際上預測可投資的次數: 6494
不可投資的次數: 21131 / 實際上不可投資的次數: 21860

test Loss: 0.619824
Accuracy: 72.97%

Type1 Correct Ratio: 46.6%
Type2 Correct Ratio: 80.8%

Model has saved in checkpoint/type_35/type_35_Model_CNN_LSTM_Target_2_save

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

Train 玻璃 Epoch: 1


100%|██████████| 95/95 [00:33<00:00,  2.85it/s]



預測可投資的次數: 3547 / 實際上預測可投資的次數: 6011
不可投資的次數: 20572 / 實際上不可投資的次數: 18108

train Loss: 0.593747
Accuracy: 74.99%

Type1 Correct Ratio: 29.33%
Type2 Correct Ratio: 90.15%

--------------------------------------------------------------------------------------------------------------------------------------------


100%|██████████| 10/10 [00:02<00:00,  3.95it/s]


預測可投資的次數: 238 / 實際上預測可投資的次數: 506
不可投資的次數: 2126 / 實際上不可投資的次數: 1858

test Loss: 0.578223
Accuracy: 78.17%

Type1 Correct Ratio: 22.53%
Type2 Correct Ratio: 93.33%

Model has saved in checkpoint/type_37/type_37_Model_CNN_LSTM_Target_2_save

